### General purpose data cleaning. 
#### If you have cleaned a file let's keep track of what we did here.

In [1]:
import pandas as pd
import numpy as np
from File_Utilities import FileTools
import DB_Utilities
DBTools = DB_Utilities.DBTools()  # instantiate the class
FileTools.MYDIR = "./data/"

### Making crosswalk for: merging on city name and state name to fips codes.

In [2]:
patents = pd.read_csv(FileTools.get_full_file_path('reduced_2009_2019.csv'),low_memory=False )

# list_of_cities that do not have a county listed.
no_county = patents[patents.county.isna()]
list_of_cities = pd.DataFrame(no_county.groupby(['state', 'city']).size()).reset_index().rename(columns={0: 'count'})
FileTools.save_df_as_csv(list_of_cities, 'list_of_cities.csv')


# THIS IS WHERE WE DID SOME GEOCODING IN ARCGIS PRO
# Geocoded on just the city and state name. 


# Bring the new crosswalk back
city_crosswalk = pd.read_csv(FileTools.get_full_file_path('crosswalk_city_geocoded.csv'),low_memory=False )
city_crosswalk['GEOID'] = city_crosswalk['STATEFP'].astype(str).str.replace(".0", "").str.zfill(2)+city_crosswalk['COUNTYFP'].astype(str).str.replace(".0", "").str.zfill(3)
city_crosswalk_final = city_crosswalk[['USER_state', 'USER_city', 'STATEFP', 'COUNTYFP', 'GEOID']].copy()
city_crosswalk_final.rename(columns={"USER_state": "state", "USER_city": "city", "STATEFP": "STATEFP", "COUNTYFP": "COUNTYFP"}, inplace=True)

FileTools.save_df_as_csv(city_crosswalk_final, 'crosswalk_city_state_name_to_city_state_fips.csv')

C:\Users\micha\AppData\Local\Temp\ipykernel_10768\1061403620.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  city_crosswalk['GEOID'] = city_crosswalk['STATEFP'].astype(str).str.replace(".0", "").str.zfill(2)+city_crosswalk['COUNTYFP'].astype(str).str.replace(".0", "").str.zfill(3)


'crosswalk_city_state_name_to_city_state_fips.csv'

## County Business Patterns 
### Pivoting tables so county is row and all predictors are columns

In [19]:
# Load in the data.
_cbp_lq_3digit_naics_all_years_df = FileTools.load_df_from_parquet('cbp_lq_3digit_naics_all_years.gzip')
_cbp_lq_3digit_naics_all_years_df.columns


Index(['fipstate', 'fipscty', 'naics', 'emp_nf', 'emp_county_naics', 'qp1_nf',
       'qp1', 'ap_nf', 'ap', 'est', 'n<5', 'n5_9', 'n10_19', 'n20_49',
       'n50_99', 'n100_249', 'n250_499', 'n500_999', 'n1000', 'n1000_1',
       'n1000_2', 'n1000_3', 'n1000_4', 'naics_level', 'county_fips',
       'emp_county', 'percent_of_county_emp', 'percent_of_state_emp',
       'emp_state_naics', 'emp_state', 'percent_of_us_emp', 'emp_us_naics',
       'emp_us', 'location_quotient_county_state',
       'location_quotient_county_us', 'year', 'censtate', 'cencty', 'empflag',
       'n1_4'],
      dtype='object')

In [28]:
all_data_cbp = pd.DataFrame()
for i in range(2010,2021): 
# for i in range(2020,2021): 
    year = str(i)
    print(f'Working on year : {year}')    

    # Filter by the year we want.
    temp_cbp_lq_3digit_naics_all_years_df = _cbp_lq_3digit_naics_all_years_df[
        (_cbp_lq_3digit_naics_all_years_df['year'] == year) 
        & (_cbp_lq_3digit_naics_all_years_df['naics_level'] == 3)
        ][[
            'county_fips','qp1', 'ap', 'est','naics','location_quotient_county_state','location_quotient_county_us']].copy()


    # Get some aggregations by county
    group_cbp_lq_3digit_naics_all_years_df = temp_cbp_lq_3digit_naics_all_years_df[['county_fips','qp1', 'ap', 'est']].groupby(['county_fips']).sum().reset_index()
    
    
    # Pivot all the naics codes up into columns
    pivot_cbp_lq_2digit_naics_all_years_df = pd.pivot_table(temp_cbp_lq_3digit_naics_all_years_df, values='location_quotient_county_us', index=['county_fips'],
                        columns=['naics'], aggfunc=np.average, fill_value=0)


    # Add the aggregations to the pivot table
    merge_cbp_lq_2digit_naics_all_years_df = pivot_cbp_lq_2digit_naics_all_years_df.merge(group_cbp_lq_3digit_naics_all_years_df, on='county_fips', how='left')
    merge_cbp_lq_2digit_naics_all_years_df['year'] = year

    # Do I need to do this?
    merge_cbp_lq_2digit_naics_all_years_df.reset_index(inplace=True)
    all_data_cbp = pd.concat([all_data_cbp, merge_cbp_lq_2digit_naics_all_years_df])

FileTools.save_df_as_parquet(all_data_cbp, 'cbp_lq_3digit_naics_all_years_PIVOT.gzip')
FileTools.save_df_as_csv(all_data_cbp, 'cbp_lq_3digit_naics_all_years_PIVOT.csv')    

Working on year : 2010
Working on year : 2011
Working on year : 2012
Working on year : 2013
Working on year : 2014
Working on year : 2015
Working on year : 2016
Working on year : 2017
Working on year : 2018
Working on year : 2019
Working on year : 2020


'cbp_lq_3digit_naics_all_years_PIVOT.csv'

In [27]:
all_data_cbp.head()

,index,county_fips,113,114,115,211,212,213,221,236,...,712,713,721,722,811,812,813,qp1,ap,est
0,0,01001,5.555412,0.000000,0.000000,0.0,1.945867,0.0,3.227681,0.681620,...,0.00000,1.281011,1.475549,2.406618,0.915081,1.075647,2.585839,40430,174710,870
1,1,01003,0.000000,0.285288,0.000000,0.0,0.000000,0.0,0.000000,1.066502,...,0.57368,1.313262,1.722986,1.721987,0.971572,0.756357,1.509355,288364,1346818,4656
2,2,01005,31.637423,0.000000,6.209217,0.0,0.000000,0.0,0.000000,0.000000,...,0.00000,0.000000,0.590252,0.721443,0.509799,0.000000,0.655996,19058,83058,514
3,3,01007,50.045967,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.822521,1.026759,1.284786,1.061995,9317,40640,292
4,4,01009,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.002452,...,0.00000,0.610964,0.000000,0.887612,2.018704,1.447656,1.658862,26070,113977,699


In [18]:
# all_data_cbp = FileTools.load_df_from_parquet('cbp_lq_3digit_naics_all_years_PIVOT.gzip')
print(all_data_cbp.shape)
all_data_cbp.head()
# DBTools.truncate_and_insert_df(all_data_cbp, "cbp_PIVOT")

(3125, 87)


naics,county_fips,113,114,115,211,212,213,221,236,237,...,623,624,711,712,713,721,722,811,812,813
0,01001,19.258521,0.000000,1.088295,0.0,5.927539,0.000000,3.084663,1.266255,1.296894,...,1.760669,0.818138,0.000000,0.000000,2.494317,1.178953,2.200564,0.931165,0.985815,2.506899
1,01003,0.434738,0.283913,0.000000,0.0,0.363784,0.019591,0.865132,1.519719,0.733203,...,0.878100,0.655100,0.200591,0.886143,1.782219,2.451151,1.839714,0.941701,0.746507,1.413468
2,01005,35.670628,0.000000,3.692662,0.0,0.000000,0.000000,2.658154,0.115809,0.000000,...,0.000000,0.082619,0.000000,0.000000,0.373166,0.551884,0.929455,0.663762,0.236436,0.853477
3,01007,17.005723,0.000000,4.849301,0.0,0.000000,0.000000,0.780958,0.133072,1.774191,...,0.000000,0.218351,0.000000,0.000000,0.000000,0.000000,0.773856,1.322033,0.257383,0.639068
4,01009,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.206961,1.420982,...,2.048118,0.692632,0.000000,0.000000,0.593446,0.000000,1.217843,1.839667,2.052264,2.033457


##

### Working on Economic Indicators

* CAINC4__ALL_AREAS_1969_2020.csv
* https://virginia.box.com/s/v70g9niz9att0381tapsnmtffksq7ahw

In [ ]:
# Read in the data, there are some encoding issues so we need to explicitly specify the encoding.
_CAINC4__ALL_AREAS_1969_2020_df = pd.read_csv(FileTools.get_full_file_path('CAINC4__ALL_AREAS_1969_2020.csv'),low_memory=False, encoding = "ISO-8859-1" )

# Clean up the description field a bit.
_CAINC4__ALL_AREAS_1969_2020_df['Description'] = _CAINC4__ALL_AREAS_1969_2020_df.Description.str.strip().str.replace("/","")

# Seems like there is double data for 'Employer contributions for government social insurance' but they have different LineCodes so we can take out one of them.
# The other descriptions are in 30's and on so we will take out 62.
_CAINC4__ALL_AREAS_1969_2020_df = _CAINC4__ALL_AREAS_1969_2020_df[~(_CAINC4__ALL_AREAS_1969_2020_df.LineCode == 62)]
# DBTools.truncate_and_insert_df(_CAINC4__ALL_AREAS_1969_2020_df, "CAINC4__ALL_AREAS_1969_2020_df")

### Pivot the table so that county is row and all predictors are columns by year

In [ ]:
all_data_ei = pd.DataFrame()
for i in range(1969,2021):  
# for i in range(1969,1970):
    year = str(i)
    print(f'Working on year : {year}')
    temp_CAINC4__ALL_AREAS_1969_2020_df = _CAINC4__ALL_AREAS_1969_2020_df[[year, 'GeoFIPS',  'Description']].copy()
    pivot_CAINC4__ALL_AREAS_1969_2020_df = pd.pivot_table(temp_CAINC4__ALL_AREAS_1969_2020_df, 
                                                    values=year
                                                    ,index=['GeoFIPS']
                                                    ,columns=['Description']
                                                    ,aggfunc=np.sum, fill_value=0)

    pivot_CAINC4__ALL_AREAS_1969_2020_df['year'] = year
    all_data_ei = pd.concat([all_data_ei, pivot_CAINC4__ALL_AREAS_1969_2020_df])   

FileTools.save_df_as_parquet(all_data_ei, 'CAINC4__ALL_AREAS_1969_2020_PIVOT.gzip')
FileTools.save_df_as_csv(all_data_ei, 'CAINC4__ALL_AREAS_1969_2020_PIVOT.csv')
DBTools.truncate_and_insert_df(all_data_ei, "CAINC4__ALL_AREAS_1969_2020_PIVOT")